In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.is_available()

True

In [6]:
'Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU')

"Setup complete. Using torch 2.2.2+cu121 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1650', major=7, minor=5, total_memory=4095MB, multi_processor_count=16)"

# Torch veriseti oluşturma

In [5]:
class KeypointsDataset(Dataset):
    # constructor method img_dir ve data_file adında 2 parametre alıyor
    # img_dir -> görüntülerin olduğu dizim
    # data_file -> veri dosyasının yolu
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(file=data_file, mode="r") as f:
            self.data = json.load(f)

        # Compose ile belirlenen sırayla ardışık işlemler uygulanır
        self.transforms = transforms.Compose(
            [
                transforms.ToPILImage(),
                # Görüntü 224x224 piksel olarak yeniden boyutlandırılır
                transforms.Resize((224, 224)),
                # PIL görüntüsü PyTorch tensörüne dönüştürülür.
                # bu dönüşüm piksel değerlerini [0,1] aralığında ölçeklendirir
                transforms.ToTensor(),
                # Bu dönüşüm, normalleştirme işlemi yapar.
                # Her bir kanalı, belirtilen ortalama (mean) ve standart sapma (std) değerlerine göre normalleştirir.
                # Bu, genellikle eğitim sürecinde modelin daha hızlı ve daha kararlı öğrenmesine yardımcı olmak için yapılır.
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        # görüntünün yüksekliğini ve genişliğini alır
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        img = self.transforms(img)
        kps = np.array(item["kps"]).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224 / w # x koordinatını yeniden boyutlandırılmış görüntü boyutlarına göre ölçeklendirir
        kps[1::2] *= 224 / h # y koordinatını yeniden boyutlandırılmış görüntü boyutlarına göre ölçeklendirir
        
        return img, kps

In [6]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
# batch_size -> her parti için 8 veri örneği kullanılarak ilerler
# shuffle -> veri seti her epoch başında rastgele karıştırılır.

# Model oluşturma

In [7]:
model = models.resnet50(pretrained=True)
# eğitilmiş ağırlıklara sahip bir ResNet-50 modeli yükler.
# pretrained=True -> ImageNet veri kümesinde eğitilmiş ağırlıkları yükler

model.fc = torch.nn.Linear(model.fc.in_features, 14*2)
# model.fc -> modelin son katmanını alır
# torch.nn.Linear -> tam bağlı bir katman oluşturur

c:\MyCode\python-projects\tennis_analysis\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\MyCode\python-projects\tennis_analysis\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = model.to(device=device)

# Modeli eğitme

In [9]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, step {i+1}/{len(train_loader)}, loss: {loss.item()}")

Epoch 1/20, step 1/829, loss: 9366.7841796875
Epoch 1/20, step 11/829, loss: 8609.7060546875
Epoch 1/20, step 21/829, loss: 8085.458984375
Epoch 1/20, step 31/829, loss: 7765.1376953125
Epoch 1/20, step 41/829, loss: 7885.95458984375
Epoch 1/20, step 51/829, loss: 7151.0654296875
Epoch 1/20, step 61/829, loss: 7205.1552734375
Epoch 1/20, step 71/829, loss: 6695.26611328125
Epoch 1/20, step 81/829, loss: 6886.0087890625
Epoch 1/20, step 91/829, loss: 6157.57958984375
Epoch 1/20, step 101/829, loss: 6284.96923828125
Epoch 1/20, step 111/829, loss: 5899.2607421875
Epoch 1/20, step 121/829, loss: 5890.24072265625
Epoch 1/20, step 131/829, loss: 5141.86181640625
Epoch 1/20, step 141/829, loss: 5127.859375
Epoch 1/20, step 151/829, loss: 4870.6337890625
Epoch 1/20, step 161/829, loss: 5002.5390625
Epoch 1/20, step 171/829, loss: 4342.29248046875
Epoch 1/20, step 181/829, loss: 3816.33935546875
Epoch 1/20, step 191/829, loss: 4105.22802734375
Epoch 1/20, step 201/829, loss: 3842.332275390625


In [7]:
torch.save(model.stat_dict(), "keypoints_model.pth")

NameError: name 'model' is not defined